In [132]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [133]:
test_df = pd.read_csv('Test.csv')
train_df = pd.read_csv('Train.csv')

In [134]:
train_df.head(5)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


## Описание набора данных

Согласно описанию, представленны обезличенные данные пользователей:
- user_id - индитификатор пользователя
- REGION - местоположение клиента
- TENURE - времени с момента заключения контракта
- MONTANT - сумма пополнения
- FREQUENCE_RECH - количество пополнений счета
- REVENUE - ежемесячный доход с клиента
- ARPU_SEGMENT - приведенный ежемесячные поступления за последние 90 дней (сумма поступлений за 90 дней / 3)
- FREQUENCE - количество пополнений счета
- DATA_VOLUME - количество подключений
- ON_NET - звонков внутри сети
- ORANGE - звонков абонентам ORANGE
- TIGO - звонков абонентам  Tigo
- ZONE1 - звонков zones1
- ZONE2 - звонков  zones2
- MRG - клиент ушел
- REGULARITY - время активности за последний 90 дней
- TOP_PACK - наиболее часто используемый пакет услуг
- FREQ_TOP_PACK - количество активаций часто используемого пакета услуг
- CHURN - пользователь ушел(целевая переменная)

## Анализ данных на предмет необходимости

In [135]:
train_df.nunique()

user_id           2154048
REGION                 14
TENURE                  8
MONTANT              6540
FREQUENCE_RECH        123
REVENUE             38114
ARPU_SEGMENT        16535
FREQUENCE              91
DATA_VOLUME         41550
ON_NET               9884
ORANGE               3167
TIGO                 1315
ZONE1                 612
ZONE2                 486
MRG                     1
REGULARITY             62
TOP_PACK              140
FREQ_TOP_PACK         245
CHURN                   2
dtype: int64

Данные в колонке *MRG* на всем объеме обучающих данных имею единственное значение - исключаем.

In [136]:
train_df = train_df.drop(columns=['MRG'])
train_df

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,NaN,NaN,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,NaN,0.0,140.0,13.0,NaN,NaN,28,All-net 500F=2000F;5d,12.0,0


In [137]:
# % пропущенных данных
train_df.isna().sum() / train_df.shape[0] * 100

user_id            0.000000
REGION            39.428044
TENURE             0.000000
MONTANT           35.131018
FREQUENCE_RECH    35.131018
REVENUE           33.706213
ARPU_SEGMENT      33.706213
FREQUENCE         33.706213
DATA_VOLUME       49.229776
ON_NET            36.520774
ORANGE            41.561191
TIGO              59.887988
ZONE1             92.120835
ZONE2             93.648052
REGULARITY         0.000000
TOP_PACK          41.902223
FREQ_TOP_PACK     41.902223
CHURN              0.000000
dtype: float64

Проанализировав процент пропущенных значений для каждого из параметров принимается следующая стратегия:
- REGION - заполнение признаком "other", возможно что заполнение каким-либо средним приведет к искажению результатов;
- MONTANT - наличие пропущенных данных в этом поле говорит, что пополнений счета не было - заполняем значением 0;
- FREQUENCE_RECH - аналогично MONTANT - заполняем значением 0;
- REVENUE - аналогично MONTANT - заполняем значением 0;
- ARPU_SEGMENT - аналогично MONTANT - заполняем значением 0;
- FREQUENCE - аналогично MONTANT - заполняем значением 0;

Следующие параметры отражают активность пользователя в специфической области:
- DATA_VOLUME - принимаем nan как отсутствие активности и присваиваем 0;
- ON_NET - принимаем nan как отсутствие активности и присваиваем 0;
- ORANGE - принимаем nan как отсутствие активности и присваиваем 0;
- TIGO - принимаем nan как отсутствие активности и присваиваем 0;
- ZONE1 - принимаем nan как отсутствие активности и присваиваем 0;
- ZONE2 - принимаем nan как отсутствие активности и присваиваем 0;

Для уменьшения вероятности потери данных, для восполнения отсутствующих значений используем метод кластеризации:
- TOP_PACK - с помощью кластеризации определим возможное значение;
- FREQ_TOP_PACK - после заполнение TOP_PACK с помощью кластеризации определим возможное значение.

In [138]:
# заполняем и переводим в int для оптимизации памяти
train_df["REGION"] = train_df["REGION"].fillna("OTHER")
train_df["MONTANT"] = train_df["MONTANT"].fillna(0)
train_df["FREQUENCE_RECH"] = train_df["FREQUENCE_RECH"].fillna(0)
train_df["REVENUE"] = train_df["REVENUE"].fillna(0)
train_df["ARPU_SEGMENT"] = train_df["ARPU_SEGMENT"].fillna(0)
train_df["FREQUENCE"] = train_df["FREQUENCE"].fillna(0)
train_df["DATA_VOLUME"] = train_df["DATA_VOLUME"].fillna(0)
train_df["ON_NET"] = train_df["ON_NET"].fillna(0)
train_df["ORANGE"] = train_df["ORANGE"].fillna(0)
train_df["TIGO"] = train_df["TIGO"].fillna(0)
train_df["ZONE1"] = train_df["ZONE1"].fillna(0)
train_df["ZONE2"] = train_df["ZONE2"].fillna(0)
train_df

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,OTHER,I 18-21 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,OTHER,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,0.0,90.0,46.0,7.0,0.0,0.0,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,0.0,0.0,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,0.0,39.0,24.0,0.0,0.0,0.0,11,Mixt 250F=Unlimited_call24H,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,OTHER,K > 24 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,NaN,NaN,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,0.0,0.0,55,"Data: 200 F=100MB,24H",9.0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,OTHER,K > 24 month,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,NaN,NaN,1
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,THIES,K > 24 month,10000.0,11.0,7120.0,2373.0,13.0,0.0,0.0,140.0,13.0,0.0,0.0,28,All-net 500F=2000F;5d,12.0,0


In [139]:
# оптимизируем типы
train_df['CHURN'] = train_df['CHURN'].astype(bool)
train_df['MONTANT'] = train_df['MONTANT'].astype(np.int32)
train_df['FREQUENCE_RECH'] = train_df['FREQUENCE_RECH'].astype(np.int16)
train_df['REVENUE'] = train_df['REVENUE'].astype(np.int32)
train_df['ARPU_SEGMENT'] = train_df['ARPU_SEGMENT'].astype(np.int32)
train_df['FREQUENCE'] = train_df['FREQUENCE'].astype(np.int16)
train_df['DATA_VOLUME'] = train_df['DATA_VOLUME'].astype(np.int32)
train_df['ON_NET'] = train_df['ON_NET'].astype(np.int32)
train_df['ORANGE'] = train_df['ORANGE'].astype(np.int16)
train_df['TIGO'] = train_df['TIGO'].astype(np.int16)
train_df['ZONE1'] = train_df['ZONE1'].astype(np.int16)
train_df['ZONE2'] = train_df['ZONE2'].astype(np.int16)
train_df['REGULARITY'] = train_df['REGULARITY'].astype(np.int16)

In [140]:
# проверяем что типы данных не "узкие", т.е. после преобразования не появилось отрицательных значений
train_df.describe()

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK
count,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,2.154048e+06,1.251454e+06
mean,3.588628e+03,7.478823e+00,3.653325e+03,1.217779e+03,9.266639e+00,1.709154e+03,1.762749e+02,5.576156e+01,9.269586e+00,6.437382e-01,4.797823e-01,2.804251e+01,9.272461e+00
std,6.307095e+03,1.202461e+01,6.405449e+03,2.135148e+03,1.366740e+01,9.628098e+03,7.080434e+02,1.636068e+02,4.182922e+01,1.176395e+01,8.638500e+00,2.228686e+01,1.228044e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+00,2.000000e+00
50%,1.000000e+03,2.000000e+00,1.000000e+03,3.330000e+02,3.000000e+00,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,5.000000e+00
75%,4.600000e+03,1.000000e+01,4.700000e+03,1.567000e+03,1.300000e+01,2.890000e+02,5.500000e+01,4.100000e+01,3.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01,1.200000e+01
max,4.700000e+05,1.330000e+02,5.321770e+05,1.773920e+05,9.100000e+01,1.823866e+06,5.080900e+04,2.132300e+04,4.174000e+03,4.792000e+03,3.697000e+03,6.200000e+01,7.130000e+02


In [141]:
# произведем One Hot кодирование категориальных переменных в столбцах 'TENURE' и 'REGION'
ten_reg_df = train_df[['user_id', "TENURE", "REGION"]]
train_df = pd.get_dummies(train_df, columns=['TENURE', 'REGION'])
train_df

,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,REGION_KEDOUGOU,REGION_KOLDA,REGION_LOUGA,REGION_MATAM,REGION_OTHER,REGION_SAINT-LOUIS,REGION_SEDHIOU,REGION_TAMBACOUNDA,REGION_THIES,REGION_ZIGUINCHOR
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,4250,15,4251,1417,17,4,388,46,1,...,0,0,0,0,0,0,0,0,0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,3600,2,1020,340,2,0,90,46,7,...,0,0,0,0,1,0,0,0,0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,13500,15,13502,4501,18,43804,41,102,2,...,0,0,0,0,0,0,0,0,0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,1000,1,985,328,1,0,39,24,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,6100,15,5800,1933,15,621,26,40,40,...,0,0,0,0,0,0,0,0,1,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,10000,11,7120,2373,13,0,0,140,13,...,0,0,0,0,0,0,0,0,1,0


In [142]:
ten_reg_df

,user_id,TENURE,REGION
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,K > 24 month,FATICK
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,I 18-21 month,OTHER
2,00001654a9d9f96303d9969d0a4a851714a4bb57,K > 24 month,OTHER
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,K > 24 month,DAKAR
4,000028d9e13a595abe061f9b58f3d76ab907850f,K > 24 month,DAKAR
...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,K > 24 month,OTHER
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,K > 24 month,THIES
2154045,fffff172fda1b4bb38a95385951908bb92379809,K > 24 month,OTHER
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,K > 24 month,THIES


In [143]:
train_df.shape

(2154048, 39)

In [144]:
# выделяем набор данных для предсказания TOP_PACK
top_pack_predict_df = train_df[train_df["TOP_PACK"].isna()].drop(columns=['TOP_PACK', 'FREQ_TOP_PACK','CHURN', 'user_id'])

In [145]:
# набор данных для обучения и теста метода KNN для дальнейшего предсказания TOP_PACK
# разделение на параметры и целевую переменную
top_pack_traintest_df = train_df[~train_df["TOP_PACK"].isna()]
top_pack_X_df = top_pack_traintest_df.drop(columns=['TOP_PACK', 'FREQ_TOP_PACK','CHURN','user_id'])
top_pack_y_df = top_pack_traintest_df['TOP_PACK']
top_pack_X_df.dtypes

MONTANT                 int32
FREQUENCE_RECH          int16
REVENUE                 int32
ARPU_SEGMENT            int32
FREQUENCE               int16
DATA_VOLUME             int32
ON_NET                  int32
ORANGE                  int16
TIGO                    int16
ZONE1                   int16
ZONE2                   int16
REGULARITY              int16
TENURE_D 3-6 month      uint8
TENURE_E 6-9 month      uint8
TENURE_F 9-12 month     uint8
TENURE_G 12-15 month    uint8
TENURE_H 15-18 month    uint8
TENURE_I 18-21 month    uint8
TENURE_J 21-24 month    uint8
TENURE_K > 24 month     uint8
REGION_DAKAR            uint8
REGION_DIOURBEL         uint8
REGION_FATICK           uint8
REGION_KAFFRINE         uint8
REGION_KAOLACK          uint8
REGION_KEDOUGOU         uint8
REGION_KOLDA            uint8
REGION_LOUGA            uint8
REGION_MATAM            uint8
REGION_OTHER            uint8
REGION_SAINT-LOUIS      uint8
REGION_SEDHIOU          uint8
REGION_TAMBACOUNDA      uint8
REGION_THI

In [146]:
top_pack_y_df

0                     On net 200F=Unlimited _call24H
2                            On-net 1000F=10MilF;10d
3                                  Data:1000F=5GB,7d
4                        Mixt 250F=Unlimited_call24H
5          MIXT:500F= 2500F on net _2500F off net;2d
                             ...                    
2154038                        All-net 500F=2000F;5d
2154040                        All-net 500F=2000F;5d
2154042                           Data:1000F=2GB,30d
2154044                        Data: 200 F=100MB,24H
2154046                        All-net 500F=2000F;5d
Name: TOP_PACK, Length: 1251454, dtype: object

In [1]:
# создаем модель для предсказания TOP_PACK методом кластеризации,
# количество кластеров == кол-ву возможных значений TOP_PACK

cluster_model1 = KMeans(n_clusters=len(train_df['TOP_PACK'].unique()))

top_pack_X_df_train, top_pack_X_df_test, top_pack_y_df_train, top_pack_y_df_test = train_test_split(top_pack_X_df,
                                                                                                    top_pack_y_df,
                                                                                                    train_size=0.7,
                                                                                                    random_state=777)

NameError: name 'KMeans' is not defined

In [148]:
# проверяем если модельку уже делали, то просто загружаем её, если нет - обучаем
try:
    cluster_model1 = pickle.load(open('TOP_PACK_predict_model.pkl', 'rb'))
except FileNotFoundError:
    print('File not found, training new model')
    cluster_model1.fit(top_pack_X_df_train)
    pickle.dump(cluster_model1, open('TOP_PACK_predict_model.pkl', 'wb'))

In [149]:
predicted_y_train = cluster_model1.predict(top_pack_X_df_train)
predicted_y_test = cluster_model1.predict(top_pack_X_df_test)

print(
    f"Результаты кластеризации:\nТренировочная выборка: {rand_score(top_pack_y_df_train, predicted_y_train)}\nТестовая выборка: {rand_score(top_pack_y_df_test, predicted_y_test)}")

Результаты кластеризации:
Тренировочная выборка: 0.8616698789026064
Тестовая выборка: 0.8619286964480138


In [150]:
# точность явно лучше случайного распределения, либо назначения среднего значения либо же моды, сохраняем модель в файл для следующего использования:
'''while True:
    answer = input('Пезезаписать файл модели? y/n')
    if str.lower(answer) == 'y\n':
        print('Saving model to the file~')
        pickle.dump(cluster_model1, open('TOP_PACK_predict_model.pkl', 'wb'))
    else:
        break'''

"while True:\n    answer = input('Пезезаписать файл модели? y/n')\n    if str.lower(answer) == 'y\n':\n        print('Saving model to the file~')\n        pickle.dump(cluster_model1, open('TOP_PACK_predict_model.pkl', 'wb'))\n    else:\n        break"

In [151]:
# заполняем отсутствующие значения "TOP_PACK"
top_pack_predict_df_values = cluster_model1.predict(top_pack_predict_df)
top_pack_labels = train_df['TOP_PACK'].unique()
top_pack_predict_df['TOP_PACK'] = top_pack_labels[top_pack_predict_df_values]
top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()
train_df['TOP_PACK'] = top_pack_y_df

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_97902/1940357785.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()


In [152]:
# проверяем на наличие каких-либо "не определенных" значений
train_df['TOP_PACK'].isna().sum()

6290

In [153]:
# удаляем неопределенности
train_df = train_df.dropna(subset=['TOP_PACK'])
train_df['TOP_PACK'].isna().sum()
train_df.to_csv('top_pack_filled.csv')

In [154]:
# необходимо осуществить кодирование TOP_PACK, однако делать это наивно не имеет смысл,
# т.к.  при One Hot кодировании получится более 140 дополнительных признаков, что не имеет смысла
# поэтому произведем поиск "уникальных" сущностей в названиях пакетов
packs = list(train_df['TOP_PACK'].unique())
packs

['On net 200F=Unlimited _call24H',
 'Data:200F=Unlimited,24H',
 'On-net 1000F=10MilF;10d',
 'Data:1000F=5GB,7d',
 'Mixt 250F=Unlimited_call24H',
 'MIXT:500F= 2500F on net _2500F off net;2d',
 'All-net 500F=2000F;5d',
 'MIXT: 500F=75(SMS, ONNET, Mo)_1000FAllNet;24h\t\t',
 'On-net 500F_FNF;3d',
 'Data: 100 F=40MB,24H',
 'MIXT: 200mnoff net _unl on net _5Go;30d',
 'Jokko_Daily',
 'Data: 200 F=100MB,24H',
 'Pilot_Youth1_290',
 'Data:490F=1GB,7d',
 'Twter_U2opia_Daily',
 'On-net 500=4000,10d',
 'Data:1000F=2GB,30d',
 'IVR Echat_Daily_50F',
 'Pilot_Youth4_490',
 'pilot_offer4',
 'All-net 500F =2000F_AllNet_Unlimited',
 'Twter_U2opia_Weekly',
 'On-net 200F=60mn;1d',
 'All-net 600F= 3000F ;5d',
 'EVC_JOKKO30',
 'MROMO_TIMWES_RENEW',
 'All-net 1000F=(3000F On+3000F Off);5d',
 'VAS(IVR_Radio_Daily)',
 'Data:3000F=10GB,30d',
 'All-net 1000=5000;5d',
 'Twter_U2opia_Monthly',
 'MIXT: 390F=04HOn-net_400SMS_400 Mo;4h\t',
 'FNF2 ( JAPPANTE)',
 'Yewouleen_PKG',
 'Data:150F=SPPackage1,24H',
 'Package3_M

In [155]:
with open('top_pack_labels.txt','w') as top_pack_labels_file:
    top_pack_labels_file.write(str(packs))

In [156]:
top_pack_cats = {'unlimited':[], 'data':[], 'mixt':[], 'net':[], 'evc':[], 'daily':[],  'week':[],  'month':[],   'pilot':[],  'gprs':[],  'wifi':[],  'internat':[],  'other':[]}
k = tuple(top_pack_cats.keys())

for pack in packs:
    if str(k[0]) in str(pack).lower():
        top_pack_cats[k[0]].append(str(pack).lower())
    elif str(k[1]) in str(pack).lower():
        top_pack_cats[k[1]].append(str(pack).lower())
    elif str(k[2]) in str(pack).lower():
        top_pack_cats[k[2]].append(str(pack).lower())
    elif str(k[3]) in str(pack).lower():
        top_pack_cats[k[3]].append(str(pack).lower())
    elif str(k[4]) in str(pack).lower():
        top_pack_cats[k[4]].append(str(pack).lower())
    elif str(k[5]) in str(pack).lower():
        top_pack_cats[k[5]].append(str(pack).lower())
    elif str(k[6]) in str(pack).lower():
        top_pack_cats[k[6]].append(str(pack).lower())
    elif str(k[7]) in str(pack).lower():
        top_pack_cats[k[7]].append(str(pack).lower())
    elif str(k[8]) in str(pack).lower():
        top_pack_cats[k[8]].append(str(pack).lower())
    elif str(k[9]) in str(pack).lower():
        top_pack_cats[k[9]].append(str(pack).lower())
    elif str(k[10]) in str(pack).lower():
        top_pack_cats[k[10]].append(str(pack).lower())
    elif str(k[11]) in str(pack).lower():
        top_pack_cats[k[11]].append(str(pack).lower())
    else:
        top_pack_cats[k[12]].append(str(pack).lower())


print(f'Разнесено названий по категориям: {str([len(top_pack_cats[_]) for _ in top_pack_cats.keys()])}')


Разнесено названий по категориям: [12, 26, 10, 23, 8, 6, 6, 8, 6, 4, 3, 3, 25]


In [157]:
# заменяем название тарифов на категории
def top_pack_coding(top_pack_value):
    global top_pack_cats
    for key in top_pack_cats.keys():
        if str(top_pack_value).lower() in top_pack_cats[key]:
            return key
train_df['TOP_PACK'] = train_df['TOP_PACK'].apply(top_pack_coding)

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_97902/34023481.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['TOP_PACK'] = train_df['TOP_PACK'].apply(top_pack_coding)


In [158]:
train_df['TOP_PACK'].value_counts()

unlimited    1138305
net           421108
data          339951
daily         100708
mixt           98244
pilot          23659
week           11436
other           7206
evc             3745
month           2201
wifi             728
internat         444
gprs              23
Name: TOP_PACK, dtype: int64

In [159]:
# One Hot кодирование полученных значений TOP_PACK
top_pack_df = train_df[['user_id','TOP_PACK']]
train_df = pd.get_dummies(train_df, columns=['TOP_PACK'])
train_df

,user_id,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_gprs,TOP_PACK_internat,TOP_PACK_mixt,TOP_PACK_month,TOP_PACK_net,TOP_PACK_other,TOP_PACK_pilot,TOP_PACK_unlimited,TOP_PACK_week,TOP_PACK_wifi
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,4250,15,4251,1417,17,4,388,46,1,...,0,0,0,0,0,0,0,1,0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,3600,2,1020,340,2,0,90,46,7,...,0,0,0,0,1,0,0,0,0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,13500,15,13502,4501,18,43804,41,102,2,...,0,0,0,0,0,0,0,0,0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,1000,1,985,328,1,0,39,24,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2154044,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,6100,15,5800,1933,15,621,26,40,40,...,0,0,0,0,0,0,0,0,0,0
2154045,fffff172fda1b4bb38a95385951908bb92379809,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2154046,fffff5911296937a37f09a37a549da2e0dad6dbb,10000,11,7120,2373,13,0,0,140,13,...,0,0,0,0,1,0,0,0,0,0


In [160]:
# TODO: реализовать аналогично подбор данных для второго проблемного столбца "FREQ_TOP_PACK"
# разделяем набор данных на известные и неизвестные
freq_top_pack_to_train = train_df[~train_df['FREQ_TOP_PACK'].isna()]
freq_top_pack_to_prediction = train_df[train_df['FREQ_TOP_PACK'].isna()]
freq_top_pack_to_train.shape, freq_top_pack_to_prediction.shape

((1251454, 51), (896304, 51))

In [161]:
# подготовка тестовой и тренировочной выборок
freq_top_pack_X_df = freq_top_pack_to_train.drop(columns=['FREQ_TOP_PACK'])
freq_top_pack_y_df = freq_top_pack_to_train['FREQ_TOP_PACK']

freq_top_pack_X_df_train, freq_top_pack_X_df_test, freq_top_pack_y_df_train, freq_top_pack_y_df_test = train_test_split(
    freq_top_pack_X_df,
    freq_top_pack_y_df,
    train_size=0.7,
    random_state=777)

In [162]:
cluster_model2 = KMeans(n_clusters=len(train_df['FREQ_TOP_PACK'].unique()))

In [163]:
# проверяем если модельку уже делали, то просто загружаем её, если нет - обучаем
try:
    cluster_model2 = pickle.load(open('TOP_PACK_FREQ_predict_model.pkl', 'rb'))
except FileNotFoundError:
    print('File not found, training new model')
    cluster_model2.fit(freq_top_pack_X_df_train.drop(columns=['user_id']), freq_top_pack_y_df_train)

In [164]:
predicted_y_train = cluster_model2.predict(freq_top_pack_X_df_train.drop(columns=['user_id']))
predicted_y_test = cluster_model2.predict(freq_top_pack_X_df_test.drop(columns=['user_id']))

print(
    f"Результаты кластеризации:\nТренировочная выборка: {rand_score(freq_top_pack_y_df_train, predicted_y_train)}\nТестовая выборка: {rand_score(freq_top_pack_y_df_test, predicted_y_test)}")

Результаты кластеризации:
Тренировочная выборка: 0.9090437116630222
Тестовая выборка: 0.9094496380833708


In [165]:
# точность явно лучше случайного распределения, либо назначения среднего значения либо же моды, сохраняем модель в файл для следующего использования:
'''while True:
    answer = input('Перезаписать файл модели? y/n')
    if str.lower(answer) == 'y\n':
        pickle.dump(cluster_model2, open('TOP_PACK_FREQ_predict_model.pkl', 'wb'))
    else:
        break'''


"while True:\n    answer = input('Перезаписать файл модели? y/n')\n    if str.lower(answer) == 'y\n':\n        pickle.dump(cluster_model2, open('TOP_PACK_FREQ_predict_model.pkl', 'wb'))\n    else:\n        break"

In [166]:
freq_top_pack_to_prediction = freq_top_pack_to_prediction.drop(columns=['FREQ_TOP_PACK'])

In [167]:
freq_top_pack_predicted =cluster_model2.predict(freq_top_pack_to_prediction.drop(columns=['user_id']))
freq_top_pack_predicted

array([  0,   0, 137, ...,   0,   0,   0], dtype=int32)

In [168]:
'''top_pack_predict_df_values = cluster_model1.predict(top_pack_predict_df)
top_pack_labels = train_df['TOP_PACK'].unique()
top_pack_predict_df['TOP_PACK'] = top_pack_labels[top_pack_predict_df_values]
top_pack_y_df = top_pack_y_df.append(top_pack_predict_df['TOP_PACK']).sort_index()
train_df['TOP_PACK'] = top_pack_y_df'''
#freq_top_pack_y_df = freq_top_pack_y_df.append(pd.Series(freq_top_pack_predicted)).sort_index()
freq_top_pack_y_df


0           8.0
2           1.0
3          11.0
4           2.0
5          18.0
           ... 
2154038     2.0
2154040     2.0
2154042     3.0
2154044     9.0
2154046    12.0
Name: FREQ_TOP_PACK, Length: 1251454, dtype: float64

In [172]:
train_df['TOP_PACK'] = top_pack_df['TOP_PACK']
train_df['REGION'] = ten_reg_df['REGION']
train_df['TENURE'] = ten_reg_df['TENURE']

train_df.drop(columns=['FREQ_TOP_PACK']).to_csv('2nd_iter.csv')